# Assignment 1. Traditional Animation

This is the start code for miniproject 1. Please refer to the [course website](http://graphics.cs.cmu.edu/nsp/course/15464-s19/www/assignments/miniProject1.htm) for instructions.

## Part I. Set up environment
- Python3.7: [https://www.anaconda.com/distribution/](https://www.anaconda.com/distribution/)
- ffmpeg: [https://www.ffmpeg.org/download.html](https://www.ffmpeg.org/download.html)

## Part II. Understand data
Sample motions are given under data folder in BVH format. BVH format has two parts, i.e. skeleton hierarchy and motion. $\bf{HIERARCHY}$ section defines joint names, joint relationship, degree of freedoms, and the order of euler angles. $\bf{MOTION}$ section defines number of frames, frame time interval, and the value for each degree of freedoms per joint per frame. Here is one simplified example of bvh file:
```
HIERARCHY
ROOT Hips
{
	OFFSET 0.000000 0.000000 0.000000
	CHANNELS 6 Xposition Yposition Zposition Zrotation Yrotation Xrotation 
	JOINT Spine
	{
		OFFSET 0.000000 10.000000 0.000000
		CHANNELS 3 Zrotation Yrotation Xrotation
		End Site
		{
			OFFSET 0.000000 10.000000 0.000000
		} 
	}
}
MOTION
Frames: 2
Frame Time: 0.033333
0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 90
```

In $\bf{HIERARCHY}$ section, we could read
- Joint names: There are three joints, "Hips", "Spine", and an end site. 
- Joint relationship (tree structure): "Hips" is the parent joint of "Spine", and "Spine" is the parent joint of the end site. 
- Degree of freedoms: "Hips" has 6 degree of freedom (i.e. 6 channels).
- Order of Euler angle: The order of rotations channels is the order or euler angle. Here, the euler angle order is ZYX, i.e. $v'=R_ZR_YR_Xv$

In $\bf{MOTION}$ section, we could read
- Number of frames: 2
- Frame time interval: 0.033333, i.e. 30 fps
- Motion data for each degree of frame per frame: There are 2 lines and each line represent one time frame. The visualization of these two frames look like

![alt text](data/Simple.jpg)

## Part III. Load and save BVH files
We have already provided start code to load and save bvh files

In [2]:
import os
import argparse
import numpy as np

import sys
sys.path.append('common')
from BVH import load, save

src_path = 'data/Samba_Dancing.bvh'
dst_path = 'output/'
os.makedirs(dst_path, exist_ok=True)

anim, joint_names, frame_time, order = load(src_path)                              # Load data from bvh file
save(os.path.join('output', 'output.bvh'), anim, joint_names, frame_time, order)   # Save data to bvh file

anim is an instance of Animation class, which defines orients, offsets, translations, and rotations. Denote the number of frames as $F$, number of joint as $J$, then the dimension for each variables are:
- orients: (J, Quaternion). Quaternion is class defined under common/Quaternions.py.
- offsets: (J, 3)
- positions: (F, J, 3)
- rotations: (F, J, Quaternion)

We can check the shape of each variable as follows:

In [3]:
anim.orients.qs.shape

(67, 4)

In [4]:
anim.offsets.shape

(67, 3)

In [5]:
anim.positions.shape

(324, 67, 3)

In [6]:
anim.rotations.qs.shape

(324, 67, 4)

In [8]:
joint_names[0:10]

['Hips',
 'Spine',
 'Spine1',
 'Spine2',
 'Neck',
 'Head',
 'HeadTop_End',
 'LeftEye',
 'RightEye',
 'LeftShoulder']

### Practice: What shall we do to read the data for "LeftShoulder"?
### Solution:

In [7]:
left_shoulder_id = joint_names.index("LeftShoulder")
print('LeftShoulder id', left_shoulder_id)
lshoulder_rotations = anim.rotations[:, left_shoulder_id]
print('Frame 0', lshoulder_rotations.qs[0])    # (w, x, y, z)
print('Frame 1', lshoulder_rotations.qs[1])

LeftShoulder id 9
Frame 0 [ 0.99726561  0.00877797 -0.05641605 -0.04691997]
Frame 1 [ 0.99738512  0.00882779 -0.06070663 -0.03820599]


## Part IV. Visualize data

You could always import BVH files in softwares such as Maya, Unity, Motion Builder. We also provided code to visualize motion data to mp4 files for an easy checkup. Make sure you have installed ffmpeg before running visualization.

In [8]:
import os
import argparse
import numpy as np

import sys
sys.path.append('common')
from Visualize import visualize_anim

name = os.path.basename(src_path).split('.')[0]
visualize_anim(anim, title=None, img_dir=os.path.join(dst_path, name), multi_view=False,
               video_path=os.path.join(dst_path, name + ".mp4"))

In [9]:
from IPython.display import Video
Video(os.path.join(dst_path, name + "_3d" + ".mp4"), embed=True)

### Reference:
- Holden, D., Komura, T., & Saito, J. (2017). Phase-functioned neural networks for character control. ACM Transactions on Graphics (TOG), 36(4), 42.
- https://research.cs.wisc.edu/graphics/Courses/cs-838-1999/Jeff/BVH.html